In [1]:
import aoc

data = aoc.read("day10.txt", to_grid=True)

# Part 1

In [ ]:
from collections import Counter


def next_step(field, current_direction):

    if field == "F":
        if current_direction == (-1, 0):
            return (0, 1)
        elif current_direction == (0, -1):
            return (1, 0)
    if field == "|":
        if current_direction == (-1, 0):
            return (-1, 0)
        elif current_direction == (1, 0):
            return (1, 0)
    if field == "-":
        if current_direction == (0, 1):


            return (0, 1)
        if current_direction == (0, -1):
            return (0, -1)
    if field == "L":
        if current_direction == (1, 0):
            return (0, 1)
        if current_direction == (0, -1):
            return (-1, 0)
    if field == "J":
        if current_direction == (1, 0):

            return (0, -1)
        if current_direction == (0, 1):


            return (-1, 0)
    if field == "7":

        if current_direction == (-1, 0):
            return (0, -1)
        if current_direction == (0, 1):
            return (1, 0)


    raise ValueError("Non-matching tile")



def find_starting_point(grid):
    n_rows = len(grid)
    n_cols = len(grid[0])
    for row in range(n_rows):


        for col in range(n_cols):
            if grid[row][col] == "S":

                return row, col
    raise RuntimeError("Could not find `S`")


def get_starting_direction(starting_row, starting_col, grid):
    size_grid = (len(grid), len(grid[0]))
    acceptable_next_tiles = {
        (0, 1): "7-J",
        (1, 0): "|LJ",
        (0, -1): "-FL",
        (-1, 0): "F|7",
    }
    for try_dir, acceptable in acceptable_next_tiles.items():
        new_row, new_col = starting_row + try_dir[0], starting_col + try_dir[1]
        if (
            not aoc.is_outside_bounds((new_row, new_col), size_grid)
            and grid[new_row][new_col] in acceptable
        ):
            return try_dir
    raise RuntimeError("Could not find acceptable starting direction")


def walk_through_loop(grid):

    row, col = find_starting_point(grid)

    direction = get_starting_direction(row, col, grid)

    field = grid[row][col]
    walked_spots = []
    while True:
        walked_spots.append((row, col))
        row += direction[0]
        col += direction[1]
        field = grid[row][col]
        if field == "S":
            return walked_spots
        direction = next_step(field, direction)



walked_path = walk_through_loop(data)
n_steps_loop = len(walked_path)
farthest_from_s = int((n_steps_loop + 1) / 2)


print(farthest_from_s)

# Part 2

In [3]:
from typing import Tuple
import copy

In [ ]:
def find_actual_value_of_S(grid, walked_path):
    cntr = Counter([grid[row][col] for row, col in walked_path])
    if cntr["F"] < cntr["J"]:
        return "F"
    if cntr["J"] < cntr["F"]:
        return "J"
    if cntr["L"] < cntr["7"]:
        return "L"
    if cntr["7"] < cntr["L"]:
        return "7"
    raise RuntimeError(f"Could not determine value of `S` for {cntr=}")


def get_inside_direction(current_direction, starts_on_bottom=False):
    if starts_on_bottom:
        seen = {(0, 1): (-1, 0), (1, 0): (0, 1), (-1, 0): (0, -1), (0, -1): (1, 0)}
    else:
        seen = {(0, 1): (1, 0), (1, 0): (0, -1), (-1, 0): (0, 1), (0, -1): (-1, 0)}
    return seen[current_direction]


def get_inside_directions(new_inside_direction, old_inside_direction):
    inside_directions = {new_inside_direction}
    if old_inside_direction != new_inside_direction:
        inside_directions.add(old_inside_direction)
        diag = (
            old_inside_direction[0] + new_inside_direction[0],
            old_inside_direction[1] + new_inside_direction[1],
        )
        inside_directions.add(diag)
    return inside_directions


def find_topleft_visted_f(grid, walked_path):
    n_rows = len(grid)
    n_cols = len(grid[0])
    for row in range(n_rows):
        for col in range(n_cols):
            if grid[row][col] == "F" and (row, col) in walked_path:
                return row, col


def find_immediate_inside_fields(
    walked_spots_set, grid, starting_direction, starting_loc
):
    grid_size = (len(grid), len(grid[0]))
    row, col = starting_loc
    direction = starting_direction
    field = grid[row][col]

    inside_direction = get_inside_direction(direction)
    inside_locs = find_inside_locs(
        (row, col), inside_direction, inside_direction, grid_size, walked_spots_set
    )

    while True:
        row += direction[0]
        col += direction[1]
        field = grid[row][col]
        loc = (row, col)
        if loc == starting_loc:
            break
        direction = next_step(field, direction)

        new_inside_direction = get_inside_direction(direction)
        new_inside_locs = find_inside_locs(
            loc, inside_direction, new_inside_direction, grid_size, walked_spots_set
        )
        inside_locs.extend(new_inside_locs)

        inside_direction = new_inside_direction
    return inside_locs


def find_inside_locs(
    loc: Tuple[int, int],
    inside_direction: Tuple[int, int],
    new_inside_direction: Tuple[int, int],
    grid_size: Tuple[int, int],
    boundary: set,
):
    """Find fields which are inside the loop next to (8-direction) the current position

    There can be multiple fields inside when turning a corner: e.g. L can return fields
    to the left, bottom left and below it.

    Parameters
    ----------
    pos : tuple
        The current position
    inside_direction : tuple
        The old direction we were moving along the boundary
    new_inside_direction : tuple
        The new direction we are moving along the boundary
    grid_size : tuple
        The size of the total grid
    boundary
        The full definition of fields of the boundary
    """
    inside_directions = get_inside_directions(new_inside_direction, inside_direction)
    inside_locs = []
    for inside_direction in inside_directions:
        inside_loc = (loc[0] + inside_direction[0], loc[1] + inside_direction[1])
        if inside_loc not in boundary:

            # A more general flood filling might just use outside bounds as a boundary
            # But in this case, since I am finding inside, it means something has gone
            # wrong (most likely defining outside and inside correctly)
            if aoc.is_outside_bounds(inside_loc, grid_size):
                msg = "This should never happen: an enclosed field outside the grid"
                raise RuntimeError(msg)
            if inside_loc == (1, 50):
                print("hmm....")
            inside_locs.append(inside_loc)
    return inside_locs


def find_enclosed_fields(starting_points: list, walked_spots_set: set):
    """Uses flood filling to find all inside fields from starting spots

    Flood-fills in 4 directions (not 8!)

    Parameters
    ----------
    starting_points : list
        The starting points to start the flood filling
    walked_spots_set : iterable
        The boundary points

    Returns
    -------
    All points inside
    """
    enclosed_fields = set()
    while starting_points:
        target = starting_points.pop()
        if target in enclosed_fields:
            continue
        enclosed_fields.add(target)
        for direction in aoc.DIRECTIONS.values():
            new_loc = target[0] + direction[0], target[1] + direction[1]
            if new_loc not in walked_spots_set:
                starting_points.append(new_loc)
    return enclosed_fields


def find_inside_fields(grid, walked_path):
    walked_spots_set = set(walked_path)

    # If we start on the topleft F, we know the starting direction and that the enclosed
    # fields are below the current direction
    # To do this, we must first modify "S" to its actual value, so we can later
    # walk through that pipe
    # .copy() doesn't work for list of lists, only copies the outer list
    modified_grid = copy.deepcopy(grid)
    true_value_s = find_actual_value_of_S(grid, walked_path)
    row_s, col_s = walked_path[0]
    modified_grid[row_s][col_s] = true_value_s

    starting_loc = find_topleft_visted_f(modified_grid, walked_spots_set)
    starting_direction = (0, 1)

    immediate_inside_locs = find_immediate_inside_fields(
        walked_spots_set, modified_grid, starting_direction, starting_loc
    )
    enclosed_fields = find_enclosed_fields(immediate_inside_locs, walked_spots_set)
    return enclosed_fields


enclosed_fields_new = find_inside_fields(data, walked_path)

print(len(enclosed_fields_new))

In [ ]:
def part2(filename):
    data = aoc.read(filename, to_grid=True)
    walked_path = walk_through_loop(data)
    enclosed_fields = find_inside_fields(data, walked_path)
    return len(enclosed_fields)


def run_tests():
    aoc.test(part2("day10_example1.txt"), 4)
    aoc.test(part2("day10_example2.txt"), 4)
    aoc.test(part2("day10_example3.txt"), 8)
    aoc.test(part2("day10_example4.txt"), 10)
    print("All tests OK!")


run_tests()

In [11]:
# TODO: write about even-odd appraoch
# TODO: write about shoelace's formula in combination with pick's theorem